# File 1:Firms' Preprocessing

Working on firm level data <br>
- This will generate patent_firm_full_clean.pkl file

In [1]:
import pandas as pd
import nltk
import numpy as np
import requests
import pickle

In [2]:
patents_new=pd.read_csv('patent_all_en_new.csv')

- appln                     The identifier of the application (with letter at the end, seems Google omits that)
- appln_id                  The (internal) patstat identifier of the application
- family_id                 The (internal) patstat identifier of the family that the application is a member of sets  - sets                      Indicate which sets (fuel-cell = 1, electric  vehicle = 2, both = 3) was matched
- direct                    Indicate whether this application was found directly (title/abstract/cpc/ipc) or      indirectly (family association)
- granted                   Whether or not the application was granted
- earliest_appln            Per family, the earliest application
- earliest_filing_date      The date of the family's earliest application
- publn                     Identifiers of publications associated with this application (*)
- publn_date                The dates of associated publications (*)
- publn_first_grant         Indicates for each publication if its the first one with granted status
- num_cpc_class             The number of CPC classes for this application
- cpc_class                 The CPC class symbols for this application
- num_ipc_class             The number of IPC classes for this application
- ipc_class                 The IPC class symbols for this application
- num_cited_publn           The number of forward citations (this applications refers to other publications)
- cited_publn_date          The dates of cited publications (**)
- cited_publn               The identifiers of cited publications (**)
- cited_appln               The identifiers of the applications  associated with cited publications (**)
- cited_appln_id            The (internal) patstat identifiers of the  applications associated with cited publications (**)
- cited_family_id           The (internal) patstat identifiers of the  family associated with cited publications (**)
- num_cited_family          The total number of distinct families  associated with cited publications (***)
- num_citing_publn          The number of backward citations (other  publications refer to this application)
- citing_publn_date         The dates of citing publications (**)
- citing_publn              The identifiers of citing publications (**)
- citing_appln              The identifiers of the applications  associated with citing publications (**)
- citing_appln_id           The (internal) patstat identifiers of the  applications associated with citing publications (**)
- citing_family_id          The (internal) patstat identifiers of the  family associated with citing publications (**)
- num_citing_family         The total number of distinct families  associated with citing publications (***)
- earliest_applt            The name of the (first) applicant of the  earliest publication of this application 
- current_applt             The name of the (first) applicant of the most recent publication of this application
- appln_title               The title of this application

>>
>> (*/**) These are one per line, in consistent order


>> (***) These seem to be the numbers that google uses to count citations. You can get this from the export if you aggregate all applications within a family and discard duplicates.


In [3]:
patents_new.head()

,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,publn,...,citing_publn_date,citing_publn,citing_appln,citing_appln_id,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr
0,WO9105114W,47158713,1239277,2,1,1,US55592190A,1990-07-19,1993-02-18,WO9303532A1,...,2002-07-11,WO0199222A3,WO0119827W,45621927,24395918,30,LUZ ELECTRIC FUEL ISRAEL LTD.,LUZ ELECTRIC FUEL ISRAEL LTD.,ELECTRICAL ENERGY SYSTEM,An electrical energy system including an elect...
1,US55592190A,51797454,1239277,2,1,1,US55592190A,1990-07-19,1992-06-09,US5121044A,...,1993-05-04\n1994-03-09\n1994-09-20\n1995-07-18...,US5208526A\nEP0557287A4\nUS5349535A\nUS5434020...,US63645090A\nEP91915464A\nUS96399792A\nUS15296...,52393850\n16916549\n54353461\n48212422\n457802...,27071024\n1239277\n25508001\n22545199\n2636904...,30,"Luz Electric Fuel Israel, Ltd.","Luz Electric Fuel Israel, Ltd.",Electrical energy system,An electrical energy system including an elect...
2,EP96927918A,17205496,1341188,4,0,0,JP4059995A,1995-02-28,1998-08-12,EP0857862A1,...,2002-01-02\n2002-01-10\n2002-08-29\n2017-12-06,EP1167708A2\nDE10029978A1\nDE10105233A1\nEP220...,EP01115227A\nDE10029978A\nDE10105233A\nEP08842...,15814252\n12936842\n12970055\n57178895,7646156\n7646156\n7672964\n40580359,31,"SUMITOMO ELECTRIC INDUSTRIES, LTD.","SUMITOMO ELECTRIC INDUSTRIES, LTD.",PARTICULATE TRAP FOR A DIESEL ENGINE,An object is to provide a particulate trap for...
3,JP4059995A,27121457,1341188,4,1,0,JP4059995A,1995-02-28,1996-09-10,JPH08229330A,...,2010-09-27\n2011-01-27\n2013-05-02\n2014-03-18...,KR20100103462A\nJP2011502743A\nJP2013078764A\n...,KR20107010572A\nJP2010531202A\nJP2012279486A\n...,328440678\n332201837\n406999716\n378212937\n41...,40580359\n40580359\n40580359\n40580359\n405803...,31,SUMITOMO ELECTRIC IND LTD,SUMITOMO ELECTRIC IND LTD,PARTICULATE TRAP FOR DIESEL ENGINE,PURPOSE: To obtain a trap by which the leakage...
4,WO9602405W,30070806,1341188,4,0,0,JP4059995A,1995-02-28,1998-03-05,WO9809059A1,...,NaN,NaN,NaN,NaN,NaN,31,"Sumitomo Electric Industries, Ltd.","Sumitomo Electric Industries, Ltd.",PARTICULATE TRAP FOR A DIESEL ENGINE,A particulate trap for a diesel engine designe...


In [4]:
#patents_new[patents_new.current_applt=='TOYOTA MOTOR CORP']

In [5]:
#ri[ri.violation == 'Speeding'].driver_gender.value_counts()

In [6]:
assignee=patents_new['current_applt']

In [7]:
# remove punctuation
assignee = patents_new['current_applt'].str.replace('[^\w\s]','')

In [8]:
assignee = patents_new['current_applt'].replace(".", "") 

In [9]:
assignee = patents_new['current_applt'].str.replace('\[.*?\]','')

In [10]:
assignee = patents_new['current_applt'].str.replace(",","")

In [11]:
assignee=patents_new['current_applt'].str.split()

In [12]:
assignee=patents_new['current_applt'].str.lower()

In [13]:
patents_new['assignee']=assignee

In [14]:
patents_new['assignee'].value_counts()

toyota motor corp                                                                                                                                3602
toyota jidosha kabushiki kaisha                                                                                                                  3543
robert bosch gmbh                                                                                                                                2653
hyundai motor company                                                                                                                            2230
honda motor co ltd                                                                                                                               1903
                                                                                                                                                 ... 
gosudarstvennoe obrazovatel'noe uchrezhdenie vysshego professional'nogo obrazovanija 'omskij gosudar

# cleaning data

In [15]:
assignee=patents_new['current_applt'].str.replace('LUZ ELECTRIC FUEL ISRAEL LTD','luz electric fuel israel ltd')

In [16]:
assignee=patents_new['current_applt'].str.replace('Luz Electric Fuel Israel Ltd','Luz Electric Fuel Israel Ltd')

### Solving toyota duplication

In [17]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota motor co ltd','toyota motor corp')

In [18]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota jidosha kabushiki kaisha','toyota motor corp')

In [19]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota motor co., ltd.','toyota motor corp')

In [20]:
patents_new['assignee']=patents_new['assignee'].str.replace('toyota motor corporation','toyota motor corp')

In [21]:
patents_new['assignee'].value_counts()

toyota motor corp                7991
robert bosch gmbh                2653
hyundai motor company            2230
honda motor co ltd               1903
nissan motor co ltd              1374
                                 ... 
toppan printing company, ltd.       1
inst francais du petrol             1
edlund; dag                         1
dae wha alloytech co.,ltd           1
nazri, gholam-abbas                 1
Name: assignee, Length: 14315, dtype: int64

### For honda

In [22]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda motor co ltd','honda motor co ltd')

In [23]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda giken kogyo kabushiki kaisha','honda motor co ltd')

In [24]:
patents_new['assignee']=patents_new['assignee'].str.replace('honda motor co., ltd.','honda motor co ltd')

In [25]:
patents_new['assignee'].value_counts()

toyota motor corp                                            7991
honda motor co ltd                                           3865
robert bosch gmbh                                            2653
hyundai motor company                                        2230
nissan motor co ltd                                          1374
                                                             ... 
motorenfabrik hatz-gmbh & co. kg                                1
iordanescu, fanel                                               1
changchun sida automotive technology consulting co., ltd.       1
launch software (shenzhen) co., ltd.                            1
yazaka corporation                                              1
Name: assignee, Length: 14313, dtype: int64

### nissan

In [26]:
patents_new['assignee']=patents_new['assignee'].str.replace('nissan motor co., ltd.','nissan motor co ltd')

In [27]:
patents_new['assignee'].value_counts()

toyota motor corp                                                                                                 7991
honda motor co ltd                                                                                                3865
robert bosch gmbh                                                                                                 2653
nissan motor co ltd                                                                                               2515
hyundai motor company                                                                                             2230
                                                                                                                  ... 
yongliang electronics co., ltd., suzhou city                                                                         1
brp-rotax gmbh & co. kg                                                                                              1
české vysoké učení technické v praze, fakulta st

### mitsubishi

In [28]:
patents_new['assignee']=patents_new['assignee'].str.replace('mitsubishi jidosha kogyo kabushiki kaisha','mitsubishi motors corp')

In [29]:
patents_new['assignee']=patents_new['assignee'].str.replace('mitsubishi electric corporation','mitsubishi electric corp')

In [30]:
patents_new['assignee']=patents_new['assignee'].str.replace('mitsubishi denki kabushiki kaisha','mitsubishi motors corp')

In [31]:
patents_new['assignee'].value_counts()

toyota motor corp                   7991
honda motor co ltd                  3865
robert bosch gmbh                   2653
nissan motor co ltd                 2515
hyundai motor company               2230
                                    ... 
plantz, basil, l.                      1
zhao shuhang                           1
microsoft corporation                  1
motorenfabrik hatz-gmbh & co. kg       1
schramme, martin                       1
Name: assignee, Length: 14308, dtype: int64

### hyundai

In [32]:
patents_new['assignee']=patents_new['assignee'].str.replace('hyundai motor company','hyundai motor co ltd')

In [33]:
patents_new['assignee']=patents_new['assignee'].str.replace('hyundai motor company','hyundai motor co ltd')

In [34]:
patents_new['assignee'].value_counts()

toyota motor corp       7991
honda motor co ltd      3865
robert bosch gmbh       2653
nissan motor co ltd     2515
hyundai motor co ltd    2447
                        ... 
zhang qingchun             1
mori, naoto                1
acena, miguel angel        1
kanzaki, takao             1
schwarze, malte            1
Name: assignee, Length: 14307, dtype: int64

In [35]:
patents_new['assignee']=patents_new['assignee'].str.replace('hitachi, ltd.','hitachi ltd')

In [36]:
patents_new['assignee'].value_counts()[:50]

toyota motor corp                                     7991
honda motor co ltd                                    3865
robert bosch gmbh                                     2653
nissan motor co ltd                                   2515
hyundai motor co ltd                                  2447
ford global technologies, llc                         1192
mitsubishi motors corp                                 875
hitachi ltd                                            715
denso corp                                             603
peugeot citroen automobiles sa                         598
mitsubishi electric corp                               474
denso corporation                                      337
daimlerchrysler ag                                     335
continental automotive gmbh                            316
siemens aktiengesellschaft                             285
gm global technology operations llc                    285
yamaha hatsudoki kabushiki kaisha                      2

### ford

In [37]:
patents_new['assignee']=patents_new['assignee'].str.replace('ford global tech llc','ford global technologies inc')

In [38]:
patents_new['assignee']=patents_new['assignee'].str.replace('ford global technologies, inc.','ford global technologies inc')

In [39]:
patents_new['assignee']=patents_new['assignee'].str.replace('ford global technologies, llc','ford global technologies inc')

In [40]:
patents_new['assignee']=patents_new['assignee'].str.replace('ford global technologies, llc','ford global technologies inc')

In [41]:
patents_new['assignee']=patents_new['assignee'].str.replace('ford global technologies llc','ford global technologies inc')

In [42]:
patents_new['assignee'].value_counts()[:50]

toyota motor corp                                     7991
honda motor co ltd                                    3865
robert bosch gmbh                                     2653
nissan motor co ltd                                   2515
hyundai motor co ltd                                  2447
ford global technologies inc                          1652
mitsubishi motors corp                                 875
hitachi ltd                                            715
denso corp                                             603
peugeot citroen automobiles sa                         598
mitsubishi electric corp                               474
denso corporation                                      337
daimlerchrysler ag                                     335
continental automotive gmbh                            316
gm global technology operations llc                    285
siemens aktiengesellschaft                             285
yamaha hatsudoki kabushiki kaisha                      2

### clearing . and ,

In [43]:
patents_new['assignee'] = patents_new['assignee'].str.replace(",","")

In [44]:
patents_new['assignee'] = patents_new['assignee'].str.replace(".","")

In [45]:
patents_new['assignee'].value_counts()[:50]

toyota motor corp                                     7997
honda motor co ltd                                    3899
robert bosch gmbh                                     2659
nissan motor co ltd                                   2596
hyundai motor co ltd                                  2532
ford global technologies inc                          1671
mitsubishi motors corp                                 899
hitachi ltd                                            759
denso corp                                             640
renault sas                                            632
peugeot citroen automobiles sa                         626
mitsubishi electric corp                               513
daimlerchrysler ag                                     338
denso corporation                                      337
continental automotive gmbh                            316
aisin aw co ltd                                        306
suzuki motor corp                                      2

### renault sas vs renault

In [46]:
patents_new['assignee']=patents_new['assignee'].str.replace('renault','renault sas')

In [47]:
patents_new['assignee']=patents_new['assignee'].str.replace('renault sas sas','renault sas')

In [48]:
patents_new['assignee'].value_counts()[:100]

toyota motor corp                   7997
honda motor co ltd                  3899
robert bosch gmbh                   2659
nissan motor co ltd                 2596
hyundai motor co ltd                2532
                                    ... 
mitsubishi fuso truck & bus corp      61
detroit diesel corporation            61
ford motor co                         60
peugeot citroën automobiles sa        60
dr ing hcf porsche ag                 59
Name: assignee, Length: 100, dtype: int64

### Removing really lower patent ones now

In [49]:
patents_new['assignee']=patents_new['assignee'].str.replace('nissan motor company limited','nissan motor co ltd')

In [50]:
patents_new['assignee']=patents_new['assignee'].str.replace('aisin seiki kabushiki kaisha','aisin aw co ltd')

In [51]:
patents_new['assignee']=patents_new['assignee'].str.replace('hyundai motor co ltd co ltd','hyundai motor co ltd')

In [52]:
patents_new.head()

,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,publn,...,citing_publn,citing_appln,citing_appln_id,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr,assignee
0,WO9105114W,47158713,1239277,2,1,1,US55592190A,1990-07-19,1993-02-18,WO9303532A1,...,WO0199222A3,WO0119827W,45621927,24395918,30,LUZ ELECTRIC FUEL ISRAEL LTD.,LUZ ELECTRIC FUEL ISRAEL LTD.,ELECTRICAL ENERGY SYSTEM,An electrical energy system including an elect...,luz electric fuel israel ltd
1,US55592190A,51797454,1239277,2,1,1,US55592190A,1990-07-19,1992-06-09,US5121044A,...,US5208526A\nEP0557287A4\nUS5349535A\nUS5434020...,US63645090A\nEP91915464A\nUS96399792A\nUS15296...,52393850\n16916549\n54353461\n48212422\n457802...,27071024\n1239277\n25508001\n22545199\n2636904...,30,"Luz Electric Fuel Israel, Ltd.","Luz Electric Fuel Israel, Ltd.",Electrical energy system,An electrical energy system including an elect...,luz electric fuel israel ltd
2,EP96927918A,17205496,1341188,4,0,0,JP4059995A,1995-02-28,1998-08-12,EP0857862A1,...,EP1167708A2\nDE10029978A1\nDE10105233A1\nEP220...,EP01115227A\nDE10029978A\nDE10105233A\nEP08842...,15814252\n12936842\n12970055\n57178895,7646156\n7646156\n7672964\n40580359,31,"SUMITOMO ELECTRIC INDUSTRIES, LTD.","SUMITOMO ELECTRIC INDUSTRIES, LTD.",PARTICULATE TRAP FOR A DIESEL ENGINE,An object is to provide a particulate trap for...,sumitomo electric industries ltd
3,JP4059995A,27121457,1341188,4,1,0,JP4059995A,1995-02-28,1996-09-10,JPH08229330A,...,KR20100103462A\nJP2011502743A\nJP2013078764A\n...,KR20107010572A\nJP2010531202A\nJP2012279486A\n...,328440678\n332201837\n406999716\n378212937\n41...,40580359\n40580359\n40580359\n40580359\n405803...,31,SUMITOMO ELECTRIC IND LTD,SUMITOMO ELECTRIC IND LTD,PARTICULATE TRAP FOR DIESEL ENGINE,PURPOSE: To obtain a trap by which the leakage...,sumitomo electric ind ltd
4,WO9602405W,30070806,1341188,4,0,0,JP4059995A,1995-02-28,1998-03-05,WO9809059A1,...,NaN,NaN,NaN,NaN,31,"Sumitomo Electric Industries, Ltd.","Sumitomo Electric Industries, Ltd.",PARTICULATE TRAP FOR A DIESEL ENGINE,A particulate trap for a diesel engine designe...,sumitomo electric industries ltd


In [53]:
import pickle
patents_new.to_pickle('patent_firm_clean.pkl')

In [54]:
patent = pd.read_pickle('patent_firm_clean.pkl')

In [55]:
patent.head()

,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,publn,...,citing_publn,citing_appln,citing_appln_id,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr,assignee
0,WO9105114W,47158713,1239277,2,1,1,US55592190A,1990-07-19,1993-02-18,WO9303532A1,...,WO0199222A3,WO0119827W,45621927,24395918,30,LUZ ELECTRIC FUEL ISRAEL LTD.,LUZ ELECTRIC FUEL ISRAEL LTD.,ELECTRICAL ENERGY SYSTEM,An electrical energy system including an elect...,luz electric fuel israel ltd
1,US55592190A,51797454,1239277,2,1,1,US55592190A,1990-07-19,1992-06-09,US5121044A,...,US5208526A\nEP0557287A4\nUS5349535A\nUS5434020...,US63645090A\nEP91915464A\nUS96399792A\nUS15296...,52393850\n16916549\n54353461\n48212422\n457802...,27071024\n1239277\n25508001\n22545199\n2636904...,30,"Luz Electric Fuel Israel, Ltd.","Luz Electric Fuel Israel, Ltd.",Electrical energy system,An electrical energy system including an elect...,luz electric fuel israel ltd
2,EP96927918A,17205496,1341188,4,0,0,JP4059995A,1995-02-28,1998-08-12,EP0857862A1,...,EP1167708A2\nDE10029978A1\nDE10105233A1\nEP220...,EP01115227A\nDE10029978A\nDE10105233A\nEP08842...,15814252\n12936842\n12970055\n57178895,7646156\n7646156\n7672964\n40580359,31,"SUMITOMO ELECTRIC INDUSTRIES, LTD.","SUMITOMO ELECTRIC INDUSTRIES, LTD.",PARTICULATE TRAP FOR A DIESEL ENGINE,An object is to provide a particulate trap for...,sumitomo electric industries ltd
3,JP4059995A,27121457,1341188,4,1,0,JP4059995A,1995-02-28,1996-09-10,JPH08229330A,...,KR20100103462A\nJP2011502743A\nJP2013078764A\n...,KR20107010572A\nJP2010531202A\nJP2012279486A\n...,328440678\n332201837\n406999716\n378212937\n41...,40580359\n40580359\n40580359\n40580359\n405803...,31,SUMITOMO ELECTRIC IND LTD,SUMITOMO ELECTRIC IND LTD,PARTICULATE TRAP FOR DIESEL ENGINE,PURPOSE: To obtain a trap by which the leakage...,sumitomo electric ind ltd
4,WO9602405W,30070806,1341188,4,0,0,JP4059995A,1995-02-28,1998-03-05,WO9809059A1,...,NaN,NaN,NaN,NaN,31,"Sumitomo Electric Industries, Ltd.","Sumitomo Electric Industries, Ltd.",PARTICULATE TRAP FOR A DIESEL ENGINE,A particulate trap for a diesel engine designe...,sumitomo electric industries ltd


In [56]:
patent['assignee'] = patent['assignee'].str.replace(",","")

In [57]:
patent['assignee'] = patent['assignee'].str.replace(".","")

In [58]:
patent['assignee'].value_counts()[:100]

toyota motor corp                 7997
honda motor co ltd                3899
nissan motor co ltd               2670
robert bosch gmbh                 2659
hyundai motor co ltd              2597
                                  ... 
ford motor co                       60
peugeot citroën automobiles sa      60
dr ing hcf porsche ag               59
fuji jukogyo kabushiki kaisha       58
byd co ltd                          58
Name: assignee, Length: 100, dtype: int64

In [59]:
p=patent

In [60]:
p.loc[p.assignee.str.contains('toyota jidosha',na=False)]

,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,publn,...,citing_publn,citing_appln,citing_appln_id,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr,assignee
5213,DE19701355A,13457689,11646668,4,1,0,JP674296A,1996-01-18,1997-07-24,DE19701355A1,...,DE19811574A1\nUS6401453B1\nDE10061685B4\nWO201...,DE19811574A\nUS66652700A\nDE10061685A\nWO20130...,13510156\n52611687\n12960865\n406380116\n33689...,7861206\n7861206\n18466322\n48536817\n44563021,16,"TOYOTA JIDOSHA K.K., TOYOTA, AICHI, JP","TOYOTA JIDOSHA K.K., TOYOTA, AICHI, JP",Catalytic converter for exhaust gases in vehic...,The catalytic converter (3) in the exhaust duc...,toyota jidosha kk toyota aichi jp
5352,JP1663492A,25858027,11921796,4,1,1,JP1663492A,1992-01-31,1993-08-20\n1999-05-10,JPH05209508A\nJP2888007B2,...,NaN,NaN,NaN,NaN,0,TOYOTA JIDOSHA KK,TOYOTA JIDOSHA KK,EXHAUST EMISSION CONTROL DEVICE FOR DIESEL ENGINE,PURPOSE:To perfectly carry out regeneration an...,toyota jidosha kk
5404,DE19901470A,13550451,12028753,12,1,1,JP2049598A,1998-01-16,1999-07-22\n2005-03-17,DE19901470A1\nDE19901470B4,...,WO0156824A1\nEP1211117A1\nWO03053730A1\nDE1020...,WO0101105W\nEP01000588A\nWO0203339W\nDE1020040...,15604711\n15801598\n9404250\n14923268\n1493936...,11457365\n26936991\n7708593\n34716556\n3463887...,116,"TOYOTA JIDOSHA K.K., TOYOTA","TOYOTA JIDOSHA K.K., TOYOTA",Drive control system for hybrid vehicle,A drive control system for a hybrid vehicle ha...,toyota jidosha kk toyota
5437,DE19505726A,13381314,12080783,8,1,1,JP2237194A,1994-02-21,1995-09-21\n2001-12-06,DE19505726A1\nDE19505726C2,...,DE19721526A1\nDE19826551C1\nDE19723790C2\nUS64...,DE19721526A\nDE19826551A\nDE19723790A\nUS71962...,13472609\n13521282\n13474257\n52996750\n130007...,7830237\n7870901\n7831615\n7870901\n7699147\n1...,41,"TOYOTA JIDOSHA K.K., TOYOTA","TOYOTA JIDOSHA K.K., TOYOTA",Control for generator in hybrid vehicle,A control unit for the generator output of a h...,toyota jidosha kk toyota
5457,JP2394992A,26243358,12124804,4,1,1,JP2394992A,1992-02-10,1993-08-31\n1998-08-27,JPH05223021A\nJP2789908B2,...,EP0825340A2\nUS5836293A\nUS5944003A\nUS5950607...,EP97113897A\nUS91024597A\nUS90833697A\nUS91024...,17229294\n54105124\n54096274\n54105111\n482513...,16643835\n16646061\n16605894\n16643835\n263564...,5,TOYOTA JIDOSHA KK,TOYOTA JIDOSHA KK,EVAPORATED FUEL DISPOSAL DEVICE FOR INTERNAL C...,PURPOSE:To provide an evaporated fuel disposal...,toyota jidosha kk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55596,CN201280074934A,439492866,49996798,4,0,1,WO2012069218W,2012-07-27,2015-04-08\n2016-10-12,CN104508265A\nCN104508265B,...,NaN,NaN,NaN,NaN,1,TOYOTA JIDOSHA KABUSHIKI KAISYA,TOYOTA JIDOSHA KABUSHIKI KAISYA,Exhaust purification device of internal combus...,"According to the present invention, a ternary ...",toyota jidosha kabushiki kaisya
55960,CN201280075525A,440372059,50182691,8,1,1,WO2012071720W,2012-08-28,2015-05-06\n2017-03-01,CN104602937A\nCN104602937B,...,CN106143471A,CN201610319380A,471996802,57208563,9,TOYOTA JIDOSHA KABUSHIKI KAISYA,TOYOTA JIDOSHA KABUSHIKI KAISYA,Drive device for hybrid vehicle,A drive device for a hybrid vehicle is provide...,toyota jidosha kabushiki kaisya
56096,CN201280075643A,440372063,50236673,8,1,0,WO2012072648W,2012-09-05,2015-05-06,CN104602939A,...,NaN,NaN,NaN,NaN,4,TOYOTA JIDOSHA KABUSHIKI KAISYA,TOYOTA JIDOSHA KABUSHIKI KAISYA,Drive device for hybrid vehicle,A drive device for a hybrid vehicle is provide...,toyota jidosha kabushiki kaisya
65730,CN201610098789A,468471102,56577308,8,0,0,JP2015035265A,2015-02-25,2016-08-31,CN105905099A,...,NaN,NaN,NaN,NaN,2,TOYOTA JIDOSHA KK,TOYOTA JIDOSHA KK,HYBRID VEHICLE,"A hybrid vehicle includes an engine, a power s...",toyota jidosha kk


In [61]:
len(p.loc[p.assignee.str.contains('toyota',na=False)])

8359

In [62]:
len(p.loc[p.assignee.str.contains('toyota jidosha',na=False)])

153

### Coming back to cleaning more

In [63]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kk toyota aichi jp','toyota motor corp')

In [64]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kk','toyota motor corp')

In [65]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kabushiki kaisya','toyota motor corp')

In [66]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kakushiki kaisha','toyota motor corp')

In [67]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kabushikikaisha','toyota motor corp')

In [68]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kabushikikaisha','toyota motor corp')

In [69]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kubishiki kaisha','toyota motor corp')

In [70]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kanushiki kaisha','toyota motor corp')

In [71]:
p['assignee']=p['assignee'].str.replace('toyota jidosha kabsuhiki kaisha','toyota motor corp')

In [72]:
p['assignee']=p['assignee'].str.replace("toyota industries corp","nissan motor co ltd")

In [73]:
p['assignee']=p['assignee'].str.replace('nissan motor','nissan motor co ltd')

In [74]:
p['assignee']=p['assignee'].str.replace("nissan motor co ltd co ltd","nissan motor co ltd")

In [75]:
p['assignee']=p['assignee'].str.replace("zhengzhou nissan automobile co ltd","nissan motor co ltd")

In [76]:
p['assignee']=p['assignee'].str.replace("nissan diesel motor co ltd","nissan motor co ltd")

In [77]:
p['assignee']=p['assignee'].str.replace("suzuki motor corporation","suzuki motor corp")

In [78]:
p['assignee']=p['assignee'].str.replace("honda giken kogyo kk","honda motor co ltd")

In [79]:
p['assignee']=p['assignee'].str.replace("honda motor coltd","honda motor co ltd")

In [80]:
p['assignee']=p['assignee'].str.replace("volkswagen aktiengesellschaft","volkswagen group")

In [81]:
p['assignee']=p['assignee'].str.replace("volkswagen ag","volkswagen group")

In [82]:
p['assignee']=p['assignee'].str.replace("volkswagen ag","volkswagen group")

In [83]:
p['assignee']=p['assignee'].str.replace("peugeot citroen automobiles sa","peugeot citroen automobiles")

In [84]:
p['assignee']=p['assignee'].str.replace("peugeot citroën automobiles sa","peugeot citroen automobiles")

In [85]:
p['assignee']=p['assignee'].str.replace("peugeot citroën automobiles sa","peugeot citroen automobiles")

In [86]:
p['assignee']=p['assignee'].str.replace("robert bosch gmbh","bosch corp")

In [87]:
p['assignee']=p['assignee'].str.replace("bosch gmbh robert","bosch corp")

In [88]:
p['assignee']=p['assignee'].str.replace("renault sas societe par actions simplifiee","renault sas")

In [89]:
p['assignee']=p['assignee'].str.replace("isuzu motors limited","isuzu motors ltd")

In [90]:
p['assignee']=p['assignee'].str.replace("aisin seiki co ltd","aisin aw co ltd")

In [91]:
p['assignee']=p['assignee'].str.replace("denso corporation","denso corp")

In [92]:
p['assignee']=p['assignee'].str.replace("nippondenso co ltd","denso corp")

In [93]:
p['assignee']=p['assignee'].str.replace("bayerische motoren werke aktiengesellschaft","bmw ag")

In [94]:
p['assignee']=p['assignee'].str.replace("bayerische motoren werke (bmw) ag","bmw ag")

In [95]:
p['assignee']=p['assignee'].str.replace("BMW AG","bmw ag")

In [96]:
p.loc[p.assignee.str.contains('bmw',na=False)]

,appln,appln_id,family_id,sets,direct,granted,earliest_appln,earliest_filing_date,publn_date,publn,...,citing_publn,citing_appln,citing_appln_id,citing_family_id,num_citing_family,earliest_applt,current_applt,appln_title,appln_abstr,assignee
333,EP90124706A,16827458,6401764,4,1,1,DE4007438A,1990-03-09,1991-09-11\n1994-02-02,EP0445407A1\nEP0445407B1,...,WO0029246A3\nDE102006045403A1,WO9903621W\nDE102006045403A,11714153\n15051545,26050197\n39134138,9,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,"Drive unit, especially for driving vehicles.","A drive assemby, in particular for driving a v...",bmw ag
667,DE4223782A,11230847,6463602,4,1,1,DE4223782A,1992-07-18,1994-01-20\n2010-05-06,DE4223782A1\nDE4223782B4,...,FR2757900A1\nEP0924418A3,FR9616213A\nEP98123457A,20536344\n333937,9499299\n18394357,2,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,Air intake control system for car IC engine - ...,The control system has a variable ratio of the...,bmw ag
706,EP93112017A,16987950,6468829,4,1,1,DE4232148A,1992-09-25,1994-03-30\n1994-08-03\n1997-04-16,EP0589176A2\nEP0589176A3\nEP0589176B1,...,WO9614561A1\nDE19818697A1\nWO0006886A1\nEP0952...,WO9401104W\nDE19818697A\nWO9901086W\nEP9910753...,43575875\n13515499\n11711704\n17378903\n411653...,20395849\n7865888\n7875529\n7865888\n19731870\...,11,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,Method for testing the tightness of a tank ins...,In a method for testing the tightness of a tan...,bmw ag
807,EP93116401A,16991809,6483087,4,1,1,DE4308570A,1993-03-18,1994-09-21\n1996-07-24,EP0616120A1\nEP0616120B1,...,NaN,NaN,NaN,NaN,0,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,Internal combustion engine with lambda control...,The valve is located in a tube section in the ...,bmw ag
826,EP94101610A,17032605,6485134,4,1,1,DE4311697A,1993-04-08,1994-10-26\n1996-07-24,EP0621152A1\nEP0621152B1,...,FR2798531A1\nFR2802592A1\nEP1112884A1\nFR28034...,FR0011389A\nFR0016709A\nEP00403476A\nFR9916641...,19006243\n19011081\n15746806\n20619711\n190181...,7921314\n34219570\n9553957\n9553957\n8861202\n...,28,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,"Drive arrangement for motor vehicles, especial...",The invention relates to a drive arrangement f...,bmw ag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63078,CN201510234117A,446595797,54336636,4,1,0,DE102014208919A,2014-05-12,2015-11-25,CN105089823A,...,NaN,NaN,NaN,NaN,1,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,Method and Device for Automatically Shutting o...,A method is provided for automatically shuttin...,bmw ag
63395,WO2015077038W,446283002,54601784,4,1,0,DE102014225360A,2014-12-10,2016-06-16,WO2016091548A1,...,NaN,NaN,NaN,NaN,0,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,LUBRICANT CONDUCTING HOUSING WITH A SEPARATION...,The invention relates to a lubricant conductin...,bmw ag
63397,US201715600217A,483402054,54601784,4,0,0,DE102014225360A,2014-12-10,2017-09-07,US2017254238A1,...,NaN,NaN,NaN,NaN,0,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,Lubricant Conducting Housing with a Separation...,A lubricant conducting member includes a lubri...,bmw ag
67098,WO2017051704W,474408669,57914967,4,1,0,DE102016201518A,2016-02-02,2017-08-10,WO2017133965A1,...,NaN,NaN,NaN,NaN,0,Bayerische Motoren Werke Aktiengesellschaft,Bayerische Motoren Werke Aktiengesellschaft,AIR CONDUCTION SYSTEM OF AN INTERNAL COMBUSTIO...,The invention relates to an air conduction sys...,bmw ag


In [97]:
p['assignee'].value_counts()[:60]

toyota motor corp                                     8125
honda motor co ltd                                    3977
nissan motor co ltd                                   2885
bosch corp                                            2770
hyundai motor co ltd                                  2597
ford global technologies inc                          1671
denso corp                                            1097
mitsubishi motors corp                                 899
renault sas                                            859
hitachi ltd                                            759
peugeot citroen automobiles                            688
suzuki motor corp                                      528
mitsubishi electric corp                               513
aisin aw co ltd                                        458
volkswagen group                                       375
daimlerchrysler ag                                     338
continental automotive gmbh                            3

### Final work on Ford & yamaha

In [98]:
p['assignee']=p['assignee'].str.replace("ford motor company","ford global technologies inc")

In [99]:
p['assignee']=p['assignee'].str.replace("kaisha yamaha motor co ltd","yamaha hatsudoki kabushiki kaisha")

In [100]:
p['assignee']=p['assignee'].str.replace("yamaha hatsudoki kabushiki kaisha","yamaha motor co ltd")

In [101]:
p['assignee']=p['assignee'].str.replace("mitsubishi electric corp","mitsubishi motors corp")

In [102]:
len(p[p.assignee =='yamaha hatsudoki kabushiki kaisha'])

0

In [103]:
len(p[p.assignee =='kaisha yamaha motor co ltd'])

0

In [104]:
len(p[p.assignee =='yamaha motor co ltd'])

466

In [105]:
len(p[p.assignee =='mitsubishi electric corp'])

0

In [106]:
len(p[p.assignee =='mitsubishi motors corp'])

1412

In [107]:
p['assignee'].value_counts()[:60]

toyota motor corp                                     8125
honda motor co ltd                                    3977
nissan motor co ltd                                   2885
bosch corp                                            2770
hyundai motor co ltd                                  2597
ford global technologies inc                          1813
mitsubishi motors corp                                1412
denso corp                                            1097
renault sas                                            859
hitachi ltd                                            759
peugeot citroen automobiles                            688
suzuki motor corp                                      528
yamaha motor co ltd                                    466
aisin aw co ltd                                        458
volkswagen group                                       375
daimlerchrysler ag                                     338
continental automotive gmbh                            3

In [108]:
p.shape

(68764, 35)

### BMW for paper

In [109]:
len(p[p.assignee =='bmw ag'])

212

In [219]:
#p.to_pickle('patent_firm_full_clean.pkl')
#p.to_csv('patent_firm_full_clean.csv')

### complete work